In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pplt
from scipy.cluster.vq import vq
import os
from os import path
from sklearn.cluster import KMeans
import random
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as pplt

In [2]:
class_path = "C:\\Users\\tanghao\\Desktop\\ADL\\"

In [3]:
## Create list for all class
def activity_list(path):
    ADL_list = os.listdir(path)
    return ADL_list

## This one has overlap function, LET"S CHANGE THE ABOVE METHOD!!!!!!!!!!!!!!!!!!!!!S
def chop_and_append_v2(path, segment, list_of_files, overlap): ## This return a array. Make sure to find a way to 
    X = overlap; class_all_segments = np.array([[0] * 96])
    for i in list_of_files:
        temp_file = np.array(pd.read_csv(os.path.join(path, i), delimiter = " ", header = None))
        T = len(temp_file); U = segment; N = int((T-X*U)/(1-X)/U)
        if (int((1-X)*U) * N + U < T): ## Here I calculate the remaining tuples and utilize them to make more vectors
            N = N+1+int((T - int((1-X)*U) * N - U)/int(U*(1-X)))
        temp_array = np.array([[0]*96]); origin = 0
        for i in range(N):
            temp_array = np.concatenate((temp_array,temp_file[origin:origin+segment,:].reshape(1,segment * 3)))
            origin = int(origin + segment * (1-X))
        class_all_segments = np.concatenate([class_all_segments, temp_array[1:,:]]) # [] and (), both work
    return class_all_segments[1:,:]
## Segmentation of Vectors
def segmentation_of_vector(class_path, seg, class_list, overlap):  #cut every txt to approprite length for segment X integer, put them into one array
    for i in class_list:
        vars()[i + '_vector'] = chop_and_append_v2(os.path.join(class_path, i), segment, os.listdir(os.path.join(class_path, i)), overlap)    
    feature = np.concatenate(
        [vars()[class_list[0] + '_vector'], vars()[class_list[1] + '_vector'], vars()[class_list[2] + '_vector'], vars()[class_list[3] + '_vector']
            , vars()[class_list[4] + '_vector'], vars()[class_list[5] + '_vector'], vars()[class_list[6] + '_vector'], vars()[class_list[7] + '_vector'],
         vars()[class_list[8] + '_vector'],
         vars()[class_list[9] + '_vector'], vars()[class_list[10] + '_vector'], vars()[class_list[11] + '_vector'],
         vars()[class_list[12] + '_vector'], vars()[class_list[13] + '_vector']])
    return feature
## K_fold
def k_fold(classpath, class_list,seed):
    all_k_fold = [[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    random.seed(seed)
    for c in range(len(class_list)):
        l = list(os.listdir(classpath + class_list[c]))
        l1 = random.sample(l,int(len(l)/3))
        l2 = random.sample(list(set(l)-set(l1)),int(len(l)/3))
        l3 = list(set(l)-set(l1)-set(l2)) 
        all_k_fold[c] = [l1,l2,l3]
    return all_k_fold 

In [4]:
seed = 1
class_list = activity_list(class_path)
kfold_list = k_fold(class_path, class_list, seed)

In [5]:
## Histogram
def generating_histogram(dictionary, class_path, category, cluster_number, class_list, segment, train_file, test_file, overlap):
    label = np.nan
    for i in range(len(class_list)):
        if (class_list[i] == category):
            label = i      
    hist_train = np.array([[0] * (cluster_number + 1)])
    hist_test = np.array([[0] * (cluster_number + 1)])
    for f in train_file:
        segments_in_file = chop_and_append_v2(os.path.join(class_path + category), segment, [f], overlap)
        segment_with_cluster_index = vq(segments_in_file, dictionary.cluster_centers_)[0]
        histogram = [0] * (cluster_number + 1)  ## This list is going to be the histogram's container later.
        for s in range(len(segment_with_cluster_index)):
            histogram[segment_with_cluster_index[s]] += 1
        histogram[cluster_number] = label
        hist_train = np.concatenate((hist_train, np.array([histogram])))
    hist_train_array = hist_train[1:,:].copy()
    for f in test_file:
        segments_in_file = chop_and_append_v2(os.path.join(class_path + category), segment, [f], overlap)
        segment_with_cluster_index = vq(segments_in_file, dictionary.cluster_centers_)[0]
        histogram = [0] * (cluster_number + 1)  ## This list is going to be the histogram's container later.
        for s in range(len(segment_with_cluster_index)):
            histogram[segment_with_cluster_index[s]] += 1
        histogram[cluster_number] = label
        hist_test = np.concatenate((hist_test, np.array([histogram])))
    hist_test_array = hist_test[1:,:].copy()    
    return hist_train_array,hist_test_array

In [6]:
# kfold_list is the k_fold of each class following the order of class_list
# testfold should be 0, 1 or 2
# np.concatenate(), the input array must be [[]] (np.array([[]])), then it can be merge, if one of them is only [], it doesn't work
def classification(dictionary, class_path, cluster_number, class_list, segment, kfold_list, overlap):
    fold = len(kfold_list[0]); Acc = 0; prediction = [[],[],[]]; Acc = []; test_label = []
    rfc = RandomForestClassifier(max_depth = 240, n_estimators = 100, random_state = 9)
    for f in range(fold):
        all_hist_train = np.array([[0] * (cluster_number + 1)])
        all_hist_test = np.array([[0] * (cluster_number + 1)])
        trainfold = list(set([0,1,2]) - set([f]));  count = 0;
        for c in range(len(class_list)):
            category = class_list[c]; test_file = kfold_list[c][f];
            train_file = list(np.concatenate((np.array(kfold_list[c][trainfold[0]]), np.array(kfold_list[c][trainfold[1]])))) 
            cat_train_hists, cat_test_hists = generating_histogram(dictionary, class_path, category, cluster_number, class_list, segment, train_file, test_file, overlap)
            all_hist_train = np.concatenate((all_hist_train, cat_train_hists))
            all_hist_test = np.concatenate((all_hist_test, cat_test_hists))
        rfc.fit(all_hist_train[1:,:cluster_number], all_hist_train[1:,cluster_number])
        predict_test = rfc.predict(all_hist_test[1:,:cluster_number])
        test_label.append(all_hist_test[1:,cluster_number])
        all_hist_test_a = all_hist_test[1:,:]
        for i in range(len(all_hist_test)-1):
            if (predict_test[i] == all_hist_test_a[i,cluster_number]):
                count+=1
        Acc.append(count/len(predict_test)); prediction[f].append(list(predict_test))
    accuracy = np.mean(np.array(Acc)); Max_index = Acc.index(max(Acc))
    print(pd.DataFrame(confusion_matrix(test_label[Max_index], prediction[Max_index][0])))
    return prediction[Max_index][0], test_label[Max_index], Max_index, accuracy

In [10]:
## calculate the experiment table
SEGMENT = [16,32,48]
OVERLAP = [0.0,0.5,0.9]
CLUSTER_NUM = [120,240,360,480,960]

PREDICT = [[],[],[]]
MAX_ACCU = [[],[],[]]
TRUTH = [[],[],[]]
AVE_ACCU = [[],[],[]]
Acc = [[],[],[]]

for i in range(len(SEGMENT)):
    for o in OVERLAP:
        for c in CLUSTER_NUM:
            f = segmentation_of_vector(class_path, SEGMENT[i], class_list, o)
            dictionary = KMeans(n_clusters = c, random_state = 0).fit(f)
            print("Segment: " + str(SEGMENT[i]) + " Overlap: " + str(o) + " Cluster_number: " + str(c))
            prediction, test_label, max_accu_fold, ave_accuracy = classification(dictionary, class_path, c, class_list, SEGMENT[i], kfold_list, o)
            PREDICT[i].append(prediction)
            TRUTH[i].append(test_label)
            MAX_ACCU[i].append(max_accu_fold)
            AVE_ACCU[i].append(ave_accuracy)

16
32


In [25]:
## Generate the average histogram
## enter the best cluster_number, segment, overlap !!!!!!!!!!!!!!!!
## Do average first, then do density, in fact both way make sense. 
ave_historgram = []
cluster_number = 120
overlap = 0.9
s = 48
f = segmentation_of_vector(class_path, s, class_list, overlap)
dictionary = KMeans(n_clusters = cluster_number, random_state = 0).fit(f)
for i in range(len(class_list)):  #range(len(class_list))
    train_file = list(np.concatenate((np.array(kfold_list[i][0]), np.array(kfold_list[i][1]))))
    test_file = kfold_list[i][2]
    hist1, hist2 = generating_histogram(dictionary, class_path, class_list[i], cluster_number, class_list, s, train_file, test_file, overlap)
    hist = np.concatenate((hist1,hist2)) ## here is the histogram for the whole activity
    mean_hist_class = np.mean(hist[:,:cluster_number], axis = 0)
    ave_historgram.append(mean_hist_class)

## Generate Histogram
ax = [0] * 14
fig = pplt.figure(figsize=(24,40))
fig.subplots_adjust(hspace = 0.5)
for i in range(len(ave_historgram)):    
    ax[i] = fig.add_subplot(7, 2, i+1)    
    ax[i].bar(np.arange(cluster_number), ave_historgram[i], align='center', alpha=0.5)
    pplt.xlabel('Cluster Center', size = 18)
    pplt.xticks(size = 16)    
    pplt.ylabel('Count', size = 18)
    pplt.yticks(size = 16)
    pplt.title(class_list[i] + ' Histogram', size = 20)    
fig.savefig('/folder/file')
pplt.show()

KeyboardInterrupt: 

In [ ]:
## Calculate the confusion matrix
# segment = 48; overlap = 0.9; K = 120
df = pd.DataFrame(confusion_matrix(TRUTH[1][14], PREDICT[1][14]))
df.columns = class_list
df.index = class_list
df

In [ ]:
## Function to generate the maximum vectors quantity from each class/file/list of file
def chop_and_append_v2(path, segment, list_of_files, overlap): ## This return a array. Make sure to find a way to 
    X = overlap; class_all_segments = np.array([[0] * 96])
    for i in list_of_files:
        temp_file = np.array(pd.read_csv(os.path.join(path, i), delimiter = " ", header = None))
        T = len(temp_file); U = segment; N = int((T-X*U)/(1-X)/U)
        if (int((1-X)*U) * N + U < T): ## Here I calculate the remaining tuples and utilize them to make more vectors
            N = N+1+int((T - int((1-X)*U) * N - U)/int(U*(1-X)))
        temp_array = np.array([[0]*96]); origin = 0
        for i in range(N):
            temp_array = np.concatenate((temp_array,temp_file[origin:origin+segment,:].reshape(1,segment * 3)))
            origin = int(origin + segment * (1-X))
        class_all_segments = np.concatenate([class_all_segments, temp_array[1:,:]]) # [] and (), both work
    return class_all_segments[1:,:]
## Segmentation of Vectors, generate vectors with above function and make them into one list for VQ
def segmentation_of_vector(class_path, seg, class_list, overlap):  #cut every txt to approprite length for segment X integer, put them into one array
    for i in class_list:
        vars()[i + '_vector'] = chop_and_append_v2(os.path.join(class_path, i), segment, os.listdir(os.path.join(class_path, i)), overlap)    
    feature = np.concatenate(
        [vars()[class_list[0] + '_vector'], vars()[class_list[1] + '_vector'], vars()[class_list[2] + '_vector'], vars()[class_list[3] + '_vector']
            , vars()[class_list[4] + '_vector'], vars()[class_list[5] + '_vector'], vars()[class_list[6] + '_vector'], vars()[class_list[7] + '_vector'],
         vars()[class_list[8] + '_vector'],
         vars()[class_list[9] + '_vector'], vars()[class_list[10] + '_vector'], vars()[class_list[11] + '_vector'],
         vars()[class_list[12] + '_vector'], vars()[class_list[13] + '_vector']])
    return feature
## K-Means
dictionary = KMeans(n_clusters = c, random_state = 0).fit(f) ## f is the list with all vectors from above function

## Histogram, this function's input require separated trainset and testset, overlap, segment
## the dictionary,file-path and category(class), class-list
def generating_histogram(dictionary, class_path, category, cluster_number, class_list, segment, train_file, test_file, overlap):
    label = np.nan
    for i in range(len(class_list)):
        if (class_list[i] == category):
            label = i      
    hist_train = np.array([[0] * (cluster_number + 1)])
    hist_test = np.array([[0] * (cluster_number + 1)])
    for f in train_file:
        segments_in_file = chop_and_append_v2(os.path.join(class_path + category), segment, [f], overlap)
        segment_with_cluster_index = vq(segments_in_file, dictionary.cluster_centers_)[0]
        histogram = [0] * (cluster_number + 1)  ## This list is going to be the histogram's container later.
        for s in range(len(segment_with_cluster_index)):
            histogram[segment_with_cluster_index[s]] += 1
        histogram[cluster_number] = label
        hist_train = np.concatenate((hist_train, np.array([histogram])))
    hist_train_array = hist_train[1:,:].copy()
    for f in test_file:
        segments_in_file = chop_and_append_v2(os.path.join(class_path + category), segment, [f], overlap)
        segment_with_cluster_index = vq(segments_in_file, dictionary.cluster_centers_)[0]
        histogram = [0] * (cluster_number + 1)  ## This list is going to be the histogram's container later.
        for s in range(len(segment_with_cluster_index)):
            histogram[segment_with_cluster_index[s]] += 1
        histogram[cluster_number] = label
        hist_test = np.concatenate((hist_test, np.array([histogram])))
    hist_test_array = hist_test[1:,:].copy()    
    return hist_train_array,hist_test_array
## use length = 48, K = 120, Overlap = 0.9
ave_historgram = []; cluster_number = 120 ;overlap = 0.9 ;s = 48
f = segmentation_of_vector(class_path, s, class_list, overlap)
dictionary = KMeans(n_clusters = cluster_number, random_state = 0).fit(f)
for i in range(len(class_list)):  #range(len(class_list))
    print(i)
    train_file = list(np.concatenate((np.array(kfold_list[i][0]), np.array(kfold_list[i][1]))))
    test_file = kfold_list[i][2]
    hist1, hist2 = generating_histogram(dictionary, class_path, class_list[i], cluster_number, class_list, s, train_file, test_file, overlap)
    hist = np.concatenate((hist1,hist2)) ## here is the histogram for the whole activity
    mean_hist_class = np.mean(hist[:,:cluster_number], axis = 0) ## Thiis is the mean histogram of each activity
    ave_historgram.append(mean_hist_class)
## Generate Histogram
ax = [0] * 14
fig = pplt.figure(figsize=(24,40))
fig.subplots_adjust(hspace = 0.5)
for i in range(len(ave_historgram)):    
    ax[i] = fig.add_subplot(7, 2, i+1)    
    ax[i].bar(np.arange(cluster_number), ave_historgram[i], align='center', alpha=0.5)
    pplt.xlabel('Cluster Center', size = 18)
    pplt.xticks(size = 16)    
    pplt.ylabel('Count', size = 18)
    pplt.yticks(size = 16)
    pplt.title(class_list[i] + ' Histogram', size = 20)    
fig.savefig('/folder/file')

# Classification
def classification(dictionary, class_path, cluster_number, class_list, segment, kfold_list, overlap):
    fold = len(kfold_list[0]); Acc = 0; prediction = [[],[],[]]; Acc = []; test_label = []
    rfc = RandomForestClassifier(max_depth = 240, n_estimators = 100, random_state = 9)
    for f in range(fold):
        all_hist_train = np.array([[0] * (cluster_number + 1)])
        all_hist_test = np.array([[0] * (cluster_number + 1)])
        trainfold = list(set([0,1,2]) - set([f]));  count = 0;
        for c in range(len(class_list)):
            category = class_list[c]; test_file = kfold_list[c][f];
            train_file = list(np.concatenate((np.array(kfold_list[c][trainfold[0]]), np.array(kfold_list[c][trainfold[1]])))) 
            cat_train_hists, cat_test_hists = generating_histogram(dictionary, class_path, category, cluster_number, class_list, segment, train_file, test_file, overlap)
            all_hist_train = np.concatenate((all_hist_train, cat_train_hists))
            all_hist_test = np.concatenate((all_hist_test, cat_test_hists))
        rfc.fit(all_hist_train[1:,:cluster_number], all_hist_train[1:,cluster_number])
        predict_test = rfc.predict(all_hist_test[1:,:cluster_number])
        test_label.append(all_hist_test[1:,cluster_number])
        all_hist_test_a = all_hist_test[1:,:]
        for i in range(len(all_hist_test)-1):
            if (predict_test[i] == all_hist_test_a[i,cluster_number]):
                count+=1
        Acc.append(count/len(predict_test)); prediction[f].append(list(predict_test))
    accuracy = np.mean(np.array(Acc)); Max_index = Acc.index(max(Acc))
    print(pd.DataFrame(confusion_matrix(test_label[Max_index], prediction[Max_index][0])))
    return prediction[Max_index][0], test_label[Max_index], Max_index, accuracy